# 12. 윤곽선 검출
- 경계선을 연결한 선
- 흑백 이미지 -> 이진화 -> 윤곽선 찾기(findCountours) -> 윤곽선 그리기(drawCountours)
- 윤곽선을 자동으로 찾아주는 기능
- 윤곽선(Contour): 같은 색상 또는 강도를 가진 영역의 경계선을 따라 그려진 곡선

## 윤곽선 그리는 방법
- 이미지에서 엣지를 찾는다(threshold 또는 canny이용)
- 윤곽선을 찾아서 데이터로 저장(findContours)
- 이미지 위에 윤곽선을 그림(drawContours)

### cv2.findContours(image, mode, method, contours[,hierarchy[,offset]]]) -> contours, hierarchy
- image : 윤곽선을 검출할 이미지,
- mode : 윤곽 검출 모드
- method : 윤곽 근사 모드
- contours : 검출된 윤곽정보
- hierarchy : 이미지 위상에 대한 정보를 포함하는 벡터
- offset : (optional) 윤곽 이동 offset, 이미지 ROI 에서 윤곽선을 추출 후에 전체 이미지 컨텍스트에서 분석해야 하는 경우 유용

# 

#### 윤곽선 찾기 모드 Contour Retrieval Mode
1. cv2.RETR_EXTERNAL
  - 가장 바깥쪽 윤곽선만 가져옴
2. cv2.RETR_LIST
  - 모든 윤곽선을 가져오지만, 계층 구조 정보는 무시함
3. cv2.RETR_TREE
  - 모든 윤곽선을 가져오며, 계층 구조도 포함
4. cv2.RETR_CCOMP
  - 모든 윤곽선을 2단계 계층 구조로 나눕


In [2]:
import cv2
horse = "../images/horse.png"

## 12-1. 윤곽선 검출 기본

In [34]:
img = cv2.imread(horse)
resize_img = cv2.resize(img, None, fx=0.8, fy=0.8)
coppied1 = resize_img.copy()
coppied2 = resize_img.copy()
combined = resize_img.copy()

# 그레이 스케일로 변환
gray1 = cv2.cvtColor(coppied1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(coppied2, cv2.COLOR_BGR2GRAY)

# 이진화
ret1, binary1 = cv2.threshold(
    gray1, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
ret2, binary2 = cv2.threshold(
    gray2, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours1, hierachy1 = cv2.findContours(
    binary1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴
countours2, hierachy2 = cv2.findContours(
    binary2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴

# 윤곽선 그리기
cv2.drawContours(coppied1, contours1, -1, (128, 255, 150), 2)
cv2.drawContours(coppied2, countours2, -1, (37, 255, 210), 2)

cv2.drawContours(combined, contours1, -1, (255, 255, 255), 2)
cv2.drawContours(combined, countours2, -1, (37, 255, 210), 2)

cv2.imshow("img", combined)
cv2.imshow("contours1", coppied1)
cv2.imshow("contours2", coppied2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 12-2. BoundingRect
- 윤곽선 경계 사각형

In [40]:
img = cv2.imread(horse)
resize_img = cv2.resize(img, None, fx=0.8, fy=0.8)
coppied1 = resize_img.copy()


# 그레이 스케일로 변환
gray1 = cv2.cvtColor(coppied1, cv2.COLOR_BGR2GRAY)

# 이진화
ret1, binary1 = cv2.threshold(
    gray1, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours1, hierachy1 = cv2.findContours(
    binary1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴

for contour in contours1:
    x, y, width, height = cv2.boundingRect(contour)
    cv2.rectangle(coppied1, (x, y), (x+width, y+height),
                  (32, 54, 146), 2, cv2.LINE_AA)


cv2.imshow("img", resize_img)
cv2.imshow("contours1", coppied1)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 12-3. contourArea
- 윤곽선의 면적을 계산

In [8]:
import cv2

img = cv2.imread("../images/vehicles.png")
coppied = img.copy()
name = "Contour"
cv2.namedWindow(name)

# 윤곽선을 찾고 : grayscale -> 이진화 -> findContours
gray = cv2.cvtColor(coppied, cv2.COLOR_BGR2GRAY)

ret, binary = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
contours, hierachy = cv2.findContours(
    binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 윤곽선의 면적이 특정 크기 이상인 윤곽선을 새로운 리스트에 저장
filtered_contours = []
MIN_AREA = 1000
for contour in contours:
    if cv2.contourArea(contour) > MIN_AREA:
        filtered_contours.append(contour)

# 윤곽선의 인덱스를 선택해서 : 트랙바를 이용 -> while
cv2.createTrackbar("index", name, 0, len(filtered_contours)-1, lambda x: x)


while True:
    coppied = img.copy()
    index = cv2.getTrackbarPos("index", name)
    # print(f"area : {cv2.contourArea(contours[index])}")

    # 해당 인덱스의 bounding rect를 추출하고 : cv2.boundingRect
    x, y, width, height = cv2.boundingRect(filtered_contours[index])

    # contours[i] -> rect
    # 화면에 사격형을 그린다 : cv2.rectangle
    cv2.rectangle(coppied, (x, y), (x+width, y+height),
                  (124, 62, 148), 2, cv2.LINE_AA)
    cv2.imshow(name, coppied)

    if cv2.waitKey(1) != -1:
        break
cv2.destroyAllWindows()

In [ ]:
# img = cv2.imread(horse)
# resize_img = cv2.resize(img, None, fx=0.8, fy=0.8)
# coppied1 = resize_img.copy()
# coppied2 = resize_img.copy()
# combined = resize_img.copy()
# coppied3 = resize_img.copy()
# coppied4 = resize_img.copy()
# # 그레이 스케일로 변환
# gray1 = cv2.cvtColor(coppied1, cv2.COLOR_BGR2GRAY)
# gray2 = cv2.cvtColor(coppied2, cv2.COLOR_BGR2GRAY)
# gray3 = cv2.cvtColor(coppied3, cv2.COLOR_BGR2BGRA)
# gray4 = cv2.cvtColor(coppied4, cv2.COLOR_BGR2BGRA)
# # 이진화
# ret1, binary1 = cv2.threshold(
#     gray1, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# ret2, binary2 = cv2.threshold(
#     gray2, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# ret3, binary3 = cv2.threshold(
#     gray3, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# ret4, binary4 = cv2.threshold(
#     gray4, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# # 윤곽선 찾기
# countours1, hierachy1 = cv2.findContours(
#     binary1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴
# countours2, hierachy2 = cv2.findContours(
#     binary2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴

# countours3, hierachy3 = cv2.findContours(
#     binary3, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴
# countours4, hierachy5 = cv2.findContours(
#     binary4, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 윤곽선과 계층구조가 나옴

# # 윤곽선 그리기
# cv2.drawContours(coppied1, countours1, -1, (128, 255, 150), 2)
# cv2.drawContours(coppied2, countours2, -1, (37, 255, 210), 2)

# cv2.drawContours(combined, countours1, -1, (255, 255, 255), 2)
# cv2.drawContours(combined, countours2, -1, (37, 255, 210), 2)

# cv2.drawContours(coppied3, countours3, -1, (255, 255, 255), 2)
# cv2.drawContours(coppied4, countours4, -1, (37, 255, 210), 2)

# cv2.imshow("img", combined)
# cv2.imshow("contours1", coppied1)
# cv2.imshow("contours2", coppied2)
# cv2.imshow("contours3", coppied3)
# cv2.imshow("countours4", coppied4)
# cv2.waitKey(0)
# cv2.destroyAllWindows()